In [2]:
#%%writefile check_action_history.py
from mouseworld import mouseworld
import time
import matplotlib.pyplot as plt
import numpy as np

num_mice = 100

# Build the model
model = mouseworld.Mouseworld(num_mice, 5, 100, 50, 100, 100)


# Prepare environment by stepping food and predators and diffusing odors
# for i in range(100) :
#     model.food_schedule.step()
#     model.predator_schedule.step()
#     model.diffuse_odor_layers_parallel(model.odor_layers)
a=time.time()
for i in range(10) :
    model.food_schedule.step()
    model.predator_schedule.step()
    model.diffuse_odor_layers(model.odor_layers)
#Run for discrete number of timesteps
b=time.time()
print(b-a)
counter = 0
myrange = 1000
# for i in range(myrange) :
#     c=time.time()
#     counter += 1
#     model.step()
#     d=time.time()
#     print('sim step : %i in %f'%(counter, d-c))
#t = np.arange(1, myrange*10 +1, 1)
#t = np.arange(1, (myrange-2)*10 +1, 1)
#Run until all mice perish
while model.num_mice > 0 :
    c=time.time()
    counter += 1
    model.step()
    d=time.time()
    print('sim step : %i in %f'%(counter, d-c))
# Gather final model and agent data
#model.mousebrain_datacollector.collect(model,model.schedule)
#mousebrain_data = model.mousebrain_datacollector.get_agent_vars_dataframe()
#mousebrain_data.to_csv('results/mousebrain_data.csv', sep='\t')
model.final_datacollector.collect(model,model.all_mice_schedule)
final_model_data = model.final_datacollector.get_model_vars_dataframe()
#final_model_data.to_csv('results/final_model_data.csv', sep='\t')
final_agent_data = model.final_datacollector.get_agent_vars_dataframe()

print(final_model_data)

for i in range(len(final_agent_data)) :
    print('Name : %s'%final_agent_data.index[i][1])
    print('Age : %i'%final_agent_data['age'][0].values[i])
    print('Generation : %i'%final_agent_data['generation'][0].values[i])
    print('Offspring : %i'%final_agent_data['num_offspring'][0].values[i])
    print('Energy : %f'%final_agent_data['energy'][0].values[i])
    print(final_agent_data['action_history'][0].values[i])
    #print(final_agent_data['possible_actions'][0].values[i])
    print(final_agent_data['primary_values'][0].values[i])
    print(final_agent_data['secondary_values'][0].values[i])
    print(final_agent_data['sensor_vector'][0].values[i])
    print(final_agent_data['sensor_position'][0].values[i])

[###                     Building... 6%                         ] ETA: 0:00:02

/home/bagjohn/anaconda3/lib/python3.6/site-packages/nengo/node.py:28: UserWarning: 'Node.size_out' is being overwritten with 'Node.size_in' since 'Node.output=None'
  warnings.warn("'Node.size_out' is being overwritten with "


Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01

/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.action_history['Distance'][self.action_history.index.max()] += distance
/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.action_history['Benefit'][self.action_history.index.max()] -= loss


sim step : 1 in 20.745767


...

sim step : 19 in 43.250773
sim step : 20 in 44.473611
sim step : 21 in 43.357126
sim step : 22 in 44.851268
sim step : 23 in 44.057564
sim step : 24 in 45.904127
sim step : 25 in 45.273079
sim step : 26 in 44.063969
sim step : 27 in 43.599766


/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.action_history['Termination'][self.action_history.index.max()] = 'Death'


sim step : 28 in 45.715096
sim step : 29 in 46.719829
sim step : 30 in 47.415696
sim step : 31 in 45.251839
sim step : 32 in 44.199198
sim step : 33 in 43.648621
sim step : 34 in 43.107670
sim step : 35 in 41.890079
sim step : 36 in 40.955686
sim step : 37 in 42.135209
sim step : 38 in 41.099159
sim step : 39 in 40.805712


/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.action_history['Termination'][self.action_history.index.max()] = 'Saturation'


sim step : 40 in 42.098965
sim step : 41 in 42.187080
sim step : 42 in 41.236051
sim step : 43 in 39.697829
sim step : 44 in 41.817419
sim step : 45 in 43.596613
sim step : 46 in 42.877865
sim step : 47 in 42.222903
sim step : 48 in 42.903128
sim step : 49 in 40.081889
sim step : 50 in 39.373958
sim step : 51 in 44.450066
sim step : 52 in 41.806998
sim step : 53 in 40.385032
sim step : 54 in 40.909393
sim step : 55 in 41.194299
sim step : 56 in 43.423105
sim step : 57 in 43.280982
sim step : 58 in 38.986417
sim step : 59 in 40.803635
sim step : 60 in 42.003324
sim step : 61 in 43.955288
sim step : 62 in 41.708052
sim step : 63 in 41.629854
sim step : 64 in 39.270154
sim step : 65 in 40.917362
sim step : 66 in 40.314713
sim step : 67 in 39.955979
sim step : 68 in 42.045858
sim step : 69 in 40.573336
sim step : 70 in 39.538854
sim step : 71 in 39.338837
sim step : 72 in 39.600612
sim step : 73 in 41.824386
sim step : 74 in 41.451325
sim step : 75 in 40.779868
sim step : 76 in 39.811506
s

In [1]:
#%%writefile check_mousebrain.py

from mouseworld import mouseworld
import time

import matplotlib.pyplot as plt
import numpy as np

# Build the model
model = mouseworld.Mouseworld(1, 5, 0, 0, 100, 100)


# Prepare environment by stepping food and predators and diffusing odors
# for i in range(100) :
#     model.food_schedule.step()
#     model.predator_schedule.step()
#     model.diffuse_odor_layers_parallel(model.odor_layers)
# for i in range(10) :
#     model.food_schedule.step()
#     model.predator_schedule.step()
#     model.diffuse_odor_layers_parallel(model.odor_layers)
#Run for discrete number of timesteps
counter = 0
myrange = 20
for i in range(myrange) :
    counter += 1
    print('sim step : %i'%counter)
    model.step()
#t = np.arange(1, myrange*10 +1, 1)
#t = np.arange(1, (myrange-2)*10 +1, 1)
# Run until all mice perish
# while model.num_mice > 0 :
#     print('sim step : %i'%counter)
#     model.step()
    
# Gather final model and agent data
model.mousebrain_datacollector.collect(model,model.schedule)
mousebrain_data = model.mousebrain_datacollector.get_agent_vars_dataframe()
#mousebrain_data.to_csv('results/mousebrain_data.csv', sep='\t')

odor = mousebrain_data['odor'][0].values[0]
state = mousebrain_data['state'][0].values[0]
approach = mousebrain_data['approach'][0].values[0]
avoid = mousebrain_data['avoid'][0].values[0]
search = mousebrain_data['search'][0].values[0]
change = mousebrain_data['change'][0].values[0]
errors0 = mousebrain_data['errors0'][0].values[0]
errors1 = mousebrain_data['errors1'][0].values[0]
errors2 = mousebrain_data['errors2'][0].values[0]

# print (type(a))
# print(a)

odor1 = [row[0] for row in odor]
odor2 = [row[1] for row in odor]

approach1 = [row[0] for row in approach]
approach2 = [row[1] for row in approach]

avoid1 = [row[0] for row in avoid]
avoid2 = [row[1] for row in avoid]

search1 = [row[0] for row in search]
search2 = [row[1] for row in search]



state0 = [row[0] for row in state]
state1 = [row[1] for row in state]
state2 = [row[2] for row in state]

data = []

data.append(odor1)
data.append(odor2)
data.append(change)
data.append(search)
data.append(errors0)
data.append(errors1)
data.append(errors2)

data.append(avoid)
data.append(approach)
# plt.plot(t, odor, 'r--', t, state, 'r--', t, avoid, 'r--',
#          t, avoid, 'r--', t, search, 'r--', t, errors0, 'r--',
#          t, errors0, 'r--', t, errors1, 'r--', t, errors2, 'r--')

# plt.plot(odor1, color='blue', odor2, color = 'red', approach1, color = 'green', approach2, color = 'black')
# plt.show()
#x = np.linspace(0, 1, 10)
for i in [3]:
    plt.plot(data[i], label='$data{i}$'.format(i=i))
plt.legend(loc='best')
plt.show()

for i in [2,4]:
    plt.plot(data[i], label='$data{i}$'.format(i=i))
plt.legend(loc='best')
plt.show()

for i in [2,5]:
    plt.plot(data[i], label='$data{i}$'.format(i=i))
plt.legend(loc='best')
plt.show()

for i in [2,6]:
    plt.plot(data[i], label='$data{i}$'.format(i=i))
plt.legend(loc='best')
plt.show()

plt.plot(odor1, 'bs', change, 'r--')
plt.show()

plt.plot(errors0, 'bs', errors1, 'r--', errors2, 'g^')
plt.show()

# plt.plot(search1, 'bs', search2, 'r--')
# plt.show()

plt.plot(approach1, 'bs', approach2, 'r--')
plt.show()

plt.plot(state0, 'bs', state1, 'r--', state2, 'g^')
plt.show()

plt.plot(odor1, 'bs', odor2, 'r--')
plt.show()

/home/bagjohn/anaconda3/lib/python3.6/site-packages/nengo/node.py:28: UserWarning: 'Node.size_out' is being overwritten with 'Node.size_in' since 'Node.output=None'
  warnings.warn("'Node.size_out' is being overwritten with "


Building finished in 0:00:01.                                                   
sim step : 1


/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.action_history['Distance'][self.action_history.index.max()] += distance


sim step : 2


/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  action_history['Duration'][action_history.index.max()] += 1


sim step : 3


KeyboardInterrupt: 